In [4]:
import tensorflow as tf
from collections import defaultdict
import gzip
import cv2 as cv
import os
import numpy as np
import random

In [2]:
# # Due to high RAM consumption when extracting features from all views together, the program was modified to take
# # one view. Re-run the program by changing view numbers (i.e 1,6,10) and path to save features of all views.
# def extract_data(filepath):
#     tr_data1 = []
#     te_data1 = []
#     tr_data_labels = []
#     te_data_labels = []
    
#     IMG_SIZE = 128
#     IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
#     vgg = tf.contrib.keras.applications.VGG16(input_shape=IMG_SHAPE, weights="imagenet", include_top=False)
#     vgg.trainable = False
#     for label in sorted(os.listdir(filepath)):
#         label_path = os.path.join(filepath, label)

#         for files in sorted(os.listdir(label_path)):
#             instances_path = os.path.join(label_path, files)

#             if str(files) == 'test':
#                 for instance in sorted(os.listdir(instances_path)):
#                     images_path = os.path.join(instances_path, instance)
#                     te_data_labels.append(label)
#                     for img in os.listdir(images_path):
#                         image_path = os.path.join(images_path, img)
#                         if int(img[0:-4]) == 10:
#                             image = cv.imread(image_path)
#                             re_image = cv.resize(image, (IMG_SIZE, IMG_SIZE))
#                             te_data1.append(re_image)
                    
#             elif str(files) == 'train':
#                 for instance in sorted(os.listdir(instances_path)):
#                     images_path = os.path.join(instances_path, instance)
#                     tr_data_labels.append(label)
#                     for img in os.listdir(images_path):
#                         image_path = os.path.join(images_path, img)
#                         if int(img[0:-4]) == 10:
#                             image = cv.imread(image_path)
#                             re_image = cv.resize(image, (IMG_SIZE, IMG_SIZE))
#                             tr_data1.append(re_image)              
                
#     tr_data1 = np.array(tr_data1, dtype="float") / 255.0
#     te_data1 = np.array(te_data1, dtype="float") / 255.0
    
#     tr_features = vgg.predict(tr_data1, batch_size=32)
#     tr_features_flatten1 = tr_features.reshape((tr_features.shape[0], 3 * 3 * 512))

#     te_features = vgg.predict(te_data1, batch_size=32)
#     te_features_flatten1 = te_features.reshape((te_features.shape[0], 3 * 3 * 512))
    
#     # data = data.reshape(data.shape[0], 50, 50, 3)
#     tr_data_labels = np.array(tr_data_labels)
#     te_data_labels = np.array(te_data_labels)
#     label_names = np.unique(tr_data_labels)
#     # data_labels = tf.one_hot(indices=data_labels, depth=10)
# #     print(features_flatten.shape)
#     return tr_features_flatten1, tr_data_labels, \
#             te_features_flatten1, te_data_labels, label_names

def extract_data(self):
    train = []
    test = []
    train_labels = []
    test_labels = []
    label_names = []
    IMG_SIZE = 128
    IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

    vgg = tf.contrib.keras.applications.VGG16(input_shape=IMG_SHAPE, weights="imagenet", include_top=False)
    vgg.trainable = False

    print("\n..........loading dataset from disk..........\n")

    for label in sorted(os.listdir(data_path)):
        label_path = os.path.join(data_path, label)

        for file in (os.listdir(label_path)):
            file_path = os.path.join(label_path, file)

            if str(file) == "train":
                for instance in sorted(os.listdir(file_path)):
                    instance_path = os.path.join(file_path, instance)
                    train_views = []

                    for img in sorted(os.listdir(instance_path)):
                        img_path = os.path.join(instance_path, img)
                        image = cv.imread(img_path, cv.IMREAD_GRAYSCALE)
                        re_image = cv.resize(image, (IMG_SIZE, IMG_SIZE))
                        train_views.append(re_image)
                    image1 = cv.merge(train_views)
                    train.append(image1)
                    train_labels.append(label)
    #                 cv.imshow("image", image)
    #                 cv.waitKey(0)
    #                 cv.destroyAllWindows()
    #                 cv.imwrite("./2.jpg", image1)
    #                 break

            elif str(file) == "test":
                for instance in sorted(os.listdir(file_path)):
                    instance_path = os.path.join(file_path, instance)
                    test_views = []

                    for img in sorted(os.listdir(instance_path)):
                        img_path = os.path.join(instance_path, img)
                        image = cv.imread(img_path, cv.IMREAD_GRAYSCALE)
                        re_image = cv.resize(image, (IMG_SIZE, IMG_SIZE))
                        test_views.append(re_image)
                    image2 = cv.merge(test_views)
                    test.append(image2)
                    test_labels.append(label)

    train = np.array(train, dtype="float") / 255.0
    train_features = vgg.predict(train, batch_size=10)
    print(train_features.shape)
    train_features_flatten = train_features.reshape((train_features.shape[0], 4 * 4 * 512))
#     train = train.reshape(train.shape[0], 128, 128, 3)
    train_labels = np.array(train_labels)

    test = np.array(test, dtype="float") / 255.0
    test_features = vgg.predict(test, batch_size=10)
    print(test_features.shape)
    test_features_flatten = test_features.reshape((test_features.shape[0], 4 * 4 * 512))
#     test = test.reshape(test.shape[0], 128, 128, 3)
    test_labels = np.array(test_labels)

    label_names = np.unique(train_labels)

    return train_features_flatten, train_labels, test_features_flatten, test_labels, label_names

In [3]:
if __name__ == "__main__":
    
    data_path = '../ModelNet40/image_dataset'
    
    np_train_path = '../ModelNet40/extracted_features/train.npy.gz'
    np_test_path = '../ModelNet40/extracted_features/test.npy.gz'
    np_train_label_path = '../ModelNet40/extracted_features/train_labels.npy'
    np_test_label_path = '../ModelNet40/extracted_features/test_labels.npy'
    np_label_names_path = '../ModelNet40/extracted_features/label_names.npy'
       
    train_data, train_labels, test_data, test_labels, label_names = extract_data(data_path)

    os.makedirs(os.path.dirname(np_train_path), exist_ok=True)

    with gzip.GzipFile(np_train_path, "w") as f:
        np.save(f, train_data)
    with gzip.GzipFile(np_test_path, "w") as f:
        np.save(f, test_data)

    np.save(np_train_label_path, train_labels)
    np.save(np_test_label_path, test_labels)
    np.save(np_label_names_path, label_names)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.

..........loading dataset from disk..........

(9843, 4, 4, 512)
(2468, 4, 4, 512)
